# Протоколы аутентификации с нулевым разглашением

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел
from Crypto.Util import number

# КГПСЧ
from Crypto.Random import random

# Понадобится в протоколе Шнорра
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Hash import SHA256

# Немного математики
import gmpy2

# Для генерации идентификаторов клиентов
import uuid

---

In [4]:
def randint(a, b) -> gmpy2.mpz:
    """
    Генерация случайного числа в отрезке [a, b] с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_a = int(a)
    native_b = int(b)
    
    return gmpy2.mpz(random.randint(native_a, native_b))


def getrandbits(n) -> gmpy2.mpz:
    """
    Генерация случайного числа длиной n бит с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_n = int(n)
    
    return gmpy2.mpz(random.getrandbits(native_n))

----

## Протокол Фиата-Шамира
![Fiat-Shamir Protocol](./images/Fiat-Shamir_2.png)

In [5]:
class FiatShamirCA(object):
    PRIME_BITS = 1024
    
    def __init__(self):
        """
        Инициализация доверенного центра.
        Генерируются два простых числа и их произведение.
        """
        
        self._db = {}
        
        self._p = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._q = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._modulus = gmpy2.mul(self._p, self._q)
        
        trace('[FiatShamirCA]', f'Modulus = {self._modulus}')
        
        
    def register_client(self, client_id: uuid.UUID, client_public_key: gmpy2.mpz) -> None:
        """
        Регистрация клиента. На вход принимает идентификатор клиента и его открытый ключ.
        Ничего не возвращает.
        Выбрасывает исключение, если клиент уже существует.
        """
        
        trace('[FiatShamirCA]', f'''Attempting to register client {client_id} 
        with public key {client_public_key}''')
        
        if client_id in self._db:
            trace('[FiatShamirCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Теперь сохраняю открытый ключ
        #
        
        self._db[client_id] = client_public_key
        trace('[FiatShamirCA]', 'Client registered successfully')
        
        
    def get_public_key(self, client_id: uuid.UUID) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента. Получает на вход идентификатор клиента.
        Возвращает открытый ключ.
        Выбрасывает исключение, если пользователь отсутствует.
        """
        
        return self._db[client_id]
       
        
    @property
    def modulus(self) -> gmpy2.mpz:
        """
        Возвращает модуль.
        """
        
        return self._modulus

In [6]:
class FiatShamirClient(object):
    def __init__(self, modulus: gmpy2.mpz, *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz] = None):
        """
        Инициализация клиента с выработкой ключевой пары и идентификатора.
        Последний параметр требуются для демонстрации неудачной аутентификации.
        """
        
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        self._r = None
        
        self._private_key = FiatShamirClient._generate_coprime(modulus)
        self._public_key = gmpy2.powmod(self._private_key, 2, modulus) if fake_data is None else fake_data[1]
        
        trace('[FiatShamirClient]', f'Client {self._id}')
        trace('[FiatShamirClient]', f'Public key = {self._public_key}')
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора.
        """
        
        return self._id
    
    
    def get_x(self, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение случайного значения, вычисляемого по формуле:
                    x = r ** 2 mod n,
        где n - модуль, r - случайное целое число из отрезка [1, n - 1].
        Значение r сохраняется.
        """
        
        self._r = randint(1, modulus - 1)
        return gmpy2.powmod(self._r, 2, modulus)
    
    
    def get_y(self, e: gmpy2.mpz, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение числа, вычисляемого по формуле:
                    y = r * s ** e mod n,
        где n - модуль, s - закрытый ключ пользователя, r - сгенерированное 
        функцией get_x внутреннее значение, e - целое число из отрезка [0, 1],
        пришедшее от проверяющего.
        """
        
        product = gmpy2.mul(self._r, gmpy2.powmod(self._private_key, e, modulus))
        return gmpy2.t_mod(product, modulus)
        
        
    @staticmethod
    def _generate_coprime(modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Генерация случайного числа из отрезка [1, n - 1], взаимно
        простого с n, где n - модуль.
        """
        
        #
        # Генерирую число до тех пор, пока оно не станет взаимно простым с модулем
        #
        
        result = randint(1, modulus - 1)
        while gmpy2.gcd(result, modulus) != 1:
            result = randint(1, modulus - 1)
            
        return result

In [7]:
class FiatShamirVerifier(object):
    def __init__(self, iterations: int):
        """
        Инициализация проверяющего. Задает коичество итераций проверки.
        """
        
        self._iterations = iterations
        
        
    def authenticate(self, client: FiatShamirClient, ca: FiatShamirCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        trace('[FiatShamirVerifier]', f'Attempt to authenticate client {client.identifier}')
        trace('[FiatShamirVerifier]', f'Number of iterations = {self._iterations}')
        
        is_successful = True
        
        for i in range(self._iterations):
            x = client.get_x(ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {x = }')
            
            e = randint(0, 1)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {e = }')
            
            y = client.get_y(e, ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {y = }')
            
            pk = ca.get_public_key(client.identifier)
            product = gmpy2.mul(x, gmpy2.powmod(pk, e, ca.modulus))
            
            successful_iteration = gmpy2.powmod(y, 2, ca.modulus) == gmpy2.t_mod(product, ca.modulus)
            is_successful &= successful_iteration
            
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] success: {successful_iteration}')
            
            if not successful_iteration:
                break
            
        print(f'Authentication successful: {is_successful}')
        return is_successful

----

In [8]:
# Создаю доверенный центр
ca = FiatShamirCA()

[TRACE] [FiatShamirCA] Modulus = 11709651578344853484218238757949990293172540317162129499198846055363622453555729581998642970012474881571887281153872575174231735877440651873005252510999065129483188228494224125059664459259727507042808263768276456888476795964316269587527452494973312989394294123276160380464435027322495054612813646107072730087531852656833603883865509813896171216720037762910483880005463888875802568374614151857632960459137568392787810004758999806430064632471269637499227071957649920601691301438727308526450970541750555357625007806037814613155311534687383085615663949717002160974886096995174195098714359855128167499748665292324646310203


In [9]:
# Создаю двух пользователей:
# - Алису - валидного пользователя
# - Еву - пользователя, представляющегося Алисой. но не знающего закрытого ключа Алисы
alice = FiatShamirClient(ca.modulus)
eve = FiatShamirClient(ca.modulus, 
                       fake_data=(alice.identifier, alice.public_key))

[TRACE] [FiatShamirClient] Client 7d55449c-05fc-4997-b816-1a63d8f13525
[TRACE] [FiatShamirClient] Public key = 6537247080128035612823072927812620015552028782783642591705635701866625292485943990016369465492111548178275266466613085329886839998505154934008852267691408469553036721353786216581962793068641577836823179138572426273249352096309659825148746530734071176218015278337908136416441441116688984816111379198996207060192766558410606614096365601484827685953014664863407924248052081280012793405738067075656412476660092979168725156742574687972532659385407427490968686184113596941622982159610500140238842513483032872040222726592130726547585230685354995411102833454698145539771373746743586489284107086020365015407522937332314247159
[TRACE] [FiatShamirClient] Client 7d55449c-05fc-4997-b816-1a63d8f13525
[TRACE] [FiatShamirClient] Public key = 65372470801280356128230729278126200155520287827836425917056357018666252924859439900163694654921115481782752664666130853298868399985051549340088522676914084695530

In [10]:
# Регистрирую валидного пользователя
ca.register_client(alice.identifier, alice.public_key)

[TRACE] [FiatShamirCA] Attempting to register client 7d55449c-05fc-4997-b816-1a63d8f13525 
        with public key 6537247080128035612823072927812620015552028782783642591705635701866625292485943990016369465492111548178275266466613085329886839998505154934008852267691408469553036721353786216581962793068641577836823179138572426273249352096309659825148746530734071176218015278337908136416441441116688984816111379198996207060192766558410606614096365601484827685953014664863407924248052081280012793405738067075656412476660092979168725156742574687972532659385407427490968686184113596941622982159610500140238842513483032872040222726592130726547585230685354995411102833454698145539771373746743586489284107086020365015407522937332314247159
[TRACE] [FiatShamirCA] Client registered successfully


In [11]:
# Для простоты будет 3 итерации
ITERATIONS = 3

# Создаю проверяющую сторону
verifier = FiatShamirVerifier(ITERATIONS)

In [12]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 7d55449c-05fc-4997-b816-1a63d8f13525
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(7601664470567402383662272778656270528626097493544147440326221612975680453328568310843222489878785904234764801294419983606140747905818200696763802329121008184065601707024777146761036231936901905326213201132959997996807680927872648191388646923404387239188230164736001410659669495082547087473884669623538031118622729688522514159989902297278776732067737261602731160040484656288203889566726312050082915181132607056365408719243314480785295986224519053010260155470662043525406079499502684447590610413898725037420477024470852476227999950830628309763714532062403991702713056061057252432495128963720208208260817698440497617848)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(756126435280363718527330160020565352162083366576112477914467935681487230794

True

In [13]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 7d55449c-05fc-4997-b816-1a63d8f13525
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(702155784044731784159615284548525625797826853140687972598108164483885874698501323276912710474470674157211029021061191288467230882122378823323695755288999813403367165391119732305681704104892403437540977116033383779954768438413768248283178773316995667590234915517476291965503749942692453097803983870046449706082008520465038437672458765993611484985615610858000797376461297854228178298252768354829436157232176665644059204099461127270322014913010174152443824990737776769016994874215226905845496261976350405287861784309911081450756476228910554087989229975702139967222604747453645484650866271641020969200349865917471013694)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(6034877200357793652569648126623641113226367307279161982929832420346111161766

False

---

## Протокол Шнорра
![Schnorr Protocol](./images/Schnorr_2.png)

In [14]:
class SchnorrCA(object):
    RSA_KEY_LENGTH = 4096
    
    def __init__(self):
        """
        Инициализация доверенного центра
        Генерация простых чисел p, q и элемента g, а также ключевой пары СА
        """
        
        self._db = {}
        
        #
        # Генерация параметров протокола
        #
        
        self._p, self._q, self._g = SchnorrCA._generate_parameters()
        
        #
        # Ключевая пара RSA для СА
        #
        
        self._private_key = RSA.generate(SchnorrCA.RSA_KEY_LENGTH)
        self._public_key = self._private_key.publickey()
        
        trace('[SchnorrCA]', f'RSA public key = {(self._public_key.e, self._public_key.n)}')
        
        
    def register_client(self, 
                        client_id: uuid.UUID, 
                        client_public_key: gmpy2.mpz) -> tuple[uuid.UUID, gmpy2.mpz, bytes]:
        """
        Регистрация клиента. Принимает на вход идентификатор клиента и его открытый ключ.
        Возвращает подписанный сервером сертификат.
        Выбрасывает исключение, если клиент уже зарегистрирован.
        """
        
        if client_id in self._db:
            trace('[SchnorrCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Создаю сертификат
        #
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        signature = pss.new(self._private_key).sign(h)
        
        certificate = (client_id, client_public_key, signature)
        trace('[SchnorrCA]', f'Certificate = {certificate}')
        
        #
        # Сохраняю его и возвращаю
        #
        
        self._db[client_id] = certificate
        return certificate
    
    
    def verify_certificate(self, certificate: tuple[uuid.UUID, gmpy2.mpz, bytes]) -> bool:
        """
        Проверка сертификата.
        Возвращает признак валидности сертификата.
        """
        
        client_id, client_public_key, signature = certificate
        
        if client_id not in self._db:
            trace('[SchnorrCA]', f"Client {client_id} doesn't exist")
            raise ValueError("Client doesn't exist")
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        verifier = pss.new(self.public_key)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа СА.
        """
        
        return self._public_key
    
    
    @property
    def parameters(self) -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Параметры протокола (p, q, g)
        """
        
        return self._p, self._q, self._g
        
        
    @staticmethod
    def _generate_parameters() -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Генерация параметров протокола (p, q, g).
        """
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = gmpy2.mpz(number.getPrime(Q_LENGTH))
        p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
        
        while not gmpy2.is_prime(p):
            p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
            
        #
        # Сгенерирую число, взаимно простое с p
        #
        
        tmp = getrandbits(P_LENGTH)
        while gmpy2.gcd(tmp, p) != 1:
            tmp = getrandbits(P_LENGTH)
        
        #
        # По малой теореме Ферма такой g будет иметь порядок q
        #
        
        g = gmpy2.powmod(tmp, (p - 1) // q, p)
        
        trace('[SchnorrCA]', f'{p = }, {q = }, {g = }')
        return p, q, g

In [15]:
class SchnorrClient(object):
    def __init__(self, parameters: tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz], *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz, tuple] = None):
        """
        Инициализация клиента. Генерирует идентификатор клиента,
        ключевую пару.
        Последний параметр необходим для демонстрации неуспешной аутентификации.
        """
        
        self._p, self._q, self._g = parameters
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        
        self._cert = None if fake_data is None else fake_data[2]
        self._r = None
        
        self._private_key = randint(1, self._q - 1)
        self._public_key  = gmpy2.powmod(
            self._g, -self._private_key, self._p) if fake_data is None else fake_data[1]
        
        trace('[SchnorrClient]', f'Client {self._id}')
        trace('[SchnorrClient]', f'Public key = {self._public_key}')
        
        
    def register(self, ca: SchnorrCA) -> None:
        """
        Регистрация клиента на СА. Принимает дескриптор СА.
        """
        
        self._cert = ca.register_client(self.identifier, self.public_key)
        
        
    def get_x(self) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = g ** r mod p
        где r - случайное целое число из отрезка [1, q - 1].
        Сохраняет значение r.
        """
        
        self._r = randint(1, self._q - 1)
        return gmpy2.powmod(self._g, self._r, self._p)
    
    
    def get_y(self, e, security_parameter) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = a * e + r mod q
        где r - сохраненное в get_x служебное значение, e - число,
        полученное от проверяющего, a - закрытый ключ клиента.
        """
        
        if e > 2 ** security_parameter or e < 1:
            trace('[SchnorrClient]', f'Invalid {e = }')
            raise ValueError('Invalid e')
            
        pre_result = gmpy2.mul(self._private_key, e) + self._r
        return gmpy2.t_mod(pre_result, self._q)
    
    
    @property
    def certificate(self):
        """
        Получение сертификата клиента.
        """
        
        return self._cert
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента.
        """
        
        return self._public_key
        
        
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id

In [16]:
class SchnorrVerifier(object):
    def __init__(self, parameters, security_parameter):
        """
        Инициализация проверяющего. Фактически только закрепляет параметры протокола.
        """
        
        self._p, self._q, self._g = parameters
        self._security_parameter = security_parameter
        
        assert security_parameter > 0 and 2 ** security_parameter < self._q
        
        self._e_values = set()
        
        
    def authenticate(self, client: SchnorrClient, ca: SchnorrCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        #
        # Получаю с клиента x и его сертификат
        #
        
        x, cert = client.get_x(), client.certificate
        trace('[SchnorrVerifier]', f'{x = }')
        
        #
        # Если сертификат битый или невалидный, то это плохо
        #
        
        if not ca.verify_certificate(cert):
            trace('[SchnorrVerifier]', f'Invalid certificate {cert}')
            return False
            
        #
        # Генерирую число е и отдаю его клиенту
        #
            
        e = randint(1, 2 ** self._security_parameter)
        while e in self._e_values:
            e = randint(1, 2 ** self._security_parameter)
            
        self._e_values.add(e)        
        trace('[SchnorrVerifier]', f'{e = }')
        
        y = client.get_y(e, self._security_parameter)
        trace('[SchnorrVerifier]', f'{y = }')
        
        #
        # Вычисляю значение, которое необходимо сравнить с полученным от клиента х
        # И сразу же сравниваю их
        # 
        
        product = gmpy2.mul(gmpy2.powmod(self._g, y, self._p), 
                            gmpy2.powmod(client.public_key, e, self._p))
        z = gmpy2.t_mod(product, self._p)
        trace('[SchnorrVerifier]', f'{z = }')
        
        is_successful = x == z
        print(f'Authentication successful: {is_successful}')
        
        return is_successful

----

In [17]:
# Создаю доверенный центр
ca = SchnorrCA()

[TRACE] [SchnorrCA] p = mpz(54768468036888540904433529078865480247270286940042556170557207727706963950553827891006510361859584842016040833889989023008973081621085883762241960161886042330870545477806070237428410501837382481012042112397025870135048288477049017028079675141222155747223936544907417944671307131560540820094983338998613384049), q = mpz(878992727688756761161853996099697554068729992679), g = mpz(50816885959646127829520087436042156813341145774570385200130938097670901018505162156726147668143734437176993190040820867470331291253345739100390324157922136970342409413488388666499600071078479126963812808561607962023910313646482622691156652646176556987519277958400088078857645641433249900236833673528585220650)
[TRACE] [SchnorrCA] RSA public key = (65537, 80014641004978038775021211433512076771085302888790418118100480680690096707826941985717995813378390332602836264836462972336233917830751285317415034796289223840176366457356357958965797722512331411405237762192287041146484050455932009284194383

In [18]:
# Создам Алису - валидного пользователя
alice = SchnorrClient(ca.parameters)

[TRACE] [SchnorrClient] Client b4bde3f8-d7e2-4211-9a63-fa9a72554a92
[TRACE] [SchnorrClient] Public key = 8588765075963210093865531560377188804711334540027067605146711948947835498051987375596707769831071446254843841054134128245898551287019845834828015601470104511044512877012213762787830713342983406831132233254149119015310938518311565679879937555957330300200454859530580544746383062083528151438828868699686698256


In [19]:
# Регистрирую валидного пользователя
alice.register(ca)

[TRACE] [SchnorrCA] Certificate = (UUID('b4bde3f8-d7e2-4211-9a63-fa9a72554a92'), mpz(8588765075963210093865531560377188804711334540027067605146711948947835498051987375596707769831071446254843841054134128245898551287019845834828015601470104511044512877012213762787830713342983406831132233254149119015310938518311565679879937555957330300200454859530580544746383062083528151438828868699686698256), b'\x93\x11-]\'xP\x04\x1b\xa6\xad\xf4q\xdf\xe4C\xec5\xfe5[\xf6\xf1a\x00F\x8c\x1e@{h\xbd\x12t\xd0}\xdf\xda%\xc8\x9e?Y;\x92^\x03\xd5\xdf\x1f\x81\x9c\xe7s\xd2e\xf1\x06\xe94\x93%A\x0b\xd5x\x85\xcd)\x98{\x93\xf9\xaa\t\xa7\xc2H\xf1\x90\xe7*\x8a\xc5\x9b0l\xf9\x86L\x0b2\x1e\x03\xc6?/\x8b\x9fR~s\x04)T\x9ew\xc7\x9b;\x913k \xb2E\xf7\xf4\xc2\xe7\x05\xe7[\xe0\xe7\x81\xb1\xdfx\x92\xc8\x82JH-T\xbdSTN\x93\x03\xb8[\x18\x0cv:\x8fp\x04\xe2\x0b\x12]\xec\x90\xce\xa5\xdb4lY\x83\xbc\x8d>-C\x8a\x9f\xb8\xac\x98\xd8\x08\xb4\x9a\x9c\xde\x870\xc7\xaf\x11\xa8\xcf\xe6\xfb\xd0\xf7\xf6\xb6$_:\xd6&V\xffqa\x1c\xd1\x08\x11\xe9\x92\x1

In [20]:
# Параметр безопасности положим равным 10 для демонстрации
security_parameter = 10

# Создаю проверяющую сторону
verifier = SchnorrVerifier(ca.parameters, security_parameter)

In [21]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [SchnorrVerifier] x = mpz(42277514516673636016750803446739962445536565322831129840542494537368107559300032445720295383155194764245154227557354632287614513609202949384160992004183158158041881461567595960884825856129721277865439319976596443259382549851017620622791169786650464498325900207260627079561995954196466696731569661650375188423)
[TRACE] [SchnorrVerifier] e = mpz(569)
[TRACE] [SchnorrVerifier] y = mpz(570281865933628183828115957376444682030209299674)
[TRACE] [SchnorrVerifier] z = mpz(42277514516673636016750803446739962445536565322831129840542494537368107559300032445720295383155194764245154227557354632287614513609202949384160992004183158158041881461567595960884825856129721277865439319976596443259382549851017620622791169786650464498325900207260627079561995954196466696731569661650375188423)
Authentication successful: True


True

In [22]:
# Создаю Еву - клиента, который пытается представиться Алисой
eve = SchnorrClient(ca.parameters, 
                    fake_data=(alice.identifier, alice.public_key, alice.certificate))

[TRACE] [SchnorrClient] Client b4bde3f8-d7e2-4211-9a63-fa9a72554a92
[TRACE] [SchnorrClient] Public key = 8588765075963210093865531560377188804711334540027067605146711948947835498051987375596707769831071446254843841054134128245898551287019845834828015601470104511044512877012213762787830713342983406831132233254149119015310938518311565679879937555957330300200454859530580544746383062083528151438828868699686698256


In [23]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [SchnorrVerifier] x = mpz(30649047366083535800526835361153243997968364642192766975813839818564962967220387405799187246759978787751913749691653692553962400127259636967750637581394108719902074930359281725000299044134038864180761015863555027221482911438741808936982170031650245446114630250671461384750508069748360577500107241301977574715)
[TRACE] [SchnorrVerifier] e = mpz(208)
[TRACE] [SchnorrVerifier] y = mpz(322267435656943090896005220366228191342668861051)
[TRACE] [SchnorrVerifier] z = mpz(2881688454444580016485700210376251293757762836751420263402587989261077371091436444034873118767307907308233674860692388485803366993840464357122888070055185612869103690386719912269016467114405117981942303125105610173842954488116918000904535692638953285742660064311553813940574701724413508042757927168952713451)
Authentication successful: False


False